In [3]:
import pandas as pd
import numpy as np
import os
import glob
from google.colab import drive
from PIL import Image
import cv2

# Mount Google Drive
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    exit()

# Define magnification-specific parameters based on typical microscopy setups
MAGNIFICATION_PARAMS = {
    'x20': {
        'pixel_size_um': 0.325,  # typical pixel size in micrometers for x20
        'field_of_view_um2': None,  # will calculate from image dimensions
        'theoretical_fov_mm2': 1.0,  # approximate field of view in mm²
    },
    'x40': {
        'pixel_size_um': 0.1625,  # typical pixel size in micrometers for x40 (half of x20)
        'field_of_view_um2': None,  # will calculate from image dimensions
        'theoretical_fov_mm2': 0.25,  # approximate field of view in mm² (1/4 of x20)
    }
}

def count_cells_in_segmentation_tif(tif_path):
    """
    Count the number of segmented cells in a TIF file.
    Assumes segmentation masks where each cell has a unique label/intensity.
    """
    try:
        # Try reading with PIL first
        img = Image.open(tif_path)
        img_array = np.array(img)

        # If it's a labeled segmentation mask, count unique non-zero labels
        unique_labels = np.unique(img_array)
        # Remove background (assuming background is 0)
        cell_labels = unique_labels[unique_labels > 0]
        cell_count = len(cell_labels)

        # Get image dimensions for area calculation
        height, width = img_array.shape[:2]

        return cell_count, width, height, img_array.dtype

    except Exception as e:
        # Fallback to OpenCV
        try:
            img_array = cv2.imread(tif_path, cv2.IMREAD_ANYDEPTH)
            if img_array is None:
                return None, None, None, None

            unique_labels = np.unique(img_array)
            cell_labels = unique_labels[unique_labels > 0]
            cell_count = len(cell_labels)

            height, width = img_array.shape[:2]
            return cell_count, width, height, img_array.dtype

        except Exception as e2:
            print(f"Error reading {tif_path}: {e}, {e2}")
            return None, None, None, None

def calculate_field_area(width, height, pixel_size_um):
    """Calculate the actual field area in μm²"""
    width_um = width * pixel_size_um
    height_um = height * pixel_size_um
    area_um2 = width_um * height_um
    return area_um2, width_um, height_um

# Define the base path and folder names
base_path = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/"
folders = ["flow3-x20", "1.4Pa-x40", "1.4Pa-x20"]

# FIXED: Remove the leading slash from subfolder_name
subfolder_name = "Cell_merged_conservative"

# Lists to store results
detailed_results = []
area_analysis = []

print("Starting magnification-aware cell counting from TIF segmentation files...")
print("\nMagnification parameters:")
for mag, params in MAGNIFICATION_PARAMS.items():
    print(f"  {mag}: pixel size = {params['pixel_size_um']} μm, theoretical FOV = {params['theoretical_fov_mm2']} mm²")

# Loop through each experimental folder
for folder in folders:
    folder_path = os.path.join(base_path, folder, subfolder_name)
    print(f"\n{'='*60}")
    print(f"Processing folder: {folder}")
    print(f"Path: {folder_path}")

    if not os.path.exists(folder_path):
        print(f"Warning: Path '{folder_path}' not found. Skipping.")
        # Let's also check what's actually in the parent directory
        parent_path = os.path.join(base_path, folder)
        if os.path.exists(parent_path):
            print(f"Contents of {parent_path}:")
            try:
                contents = os.listdir(parent_path)
                for item in contents:
                    print(f"  - {item}")
            except Exception as e:
                print(f"  Could not list contents: {e}")
        continue

    # Find all TIF files
    tif_files = glob.glob(os.path.join(folder_path, "*.tif")) + glob.glob(os.path.join(folder_path, "*.tiff"))

    if not tif_files:
        print(f"No TIF files found in {folder_path}")
        # List what files are actually there
        try:
            all_files = os.listdir(folder_path)
            print(f"Files found in directory: {all_files}")
        except Exception as e:
            print(f"Could not list files: {e}")
        continue

    print(f"Found {len(tif_files)} TIF file(s)")

    # Extract experiment parameters
    if "flow3" in folder:
        pressure = '0Pa'
        experiment = 'flow3'
    elif "1.4Pa" in folder:
        pressure = '1.4Pa'
        experiment = '1.4Pa'
    else:
        pressure = 'Unknown'
        experiment = 'Unknown'

    if "x40" in folder:
        magnification = 'x40'
    elif "x20" in folder:
        magnification = 'x20'
    else:
        magnification = 'unknown'

    # Get magnification parameters
    mag_params = MAGNIFICATION_PARAMS.get(magnification, {})
    pixel_size_um = mag_params.get('pixel_size_um', 1.0)

    total_cells_in_folder = 0
    total_area_um2 = 0

    # Process each TIF file
    for tif_file in tif_files:
        file_name = os.path.basename(tif_file)
        print(f"\nProcessing: {file_name}")

        cell_count, width, height, dtype = count_cells_in_segmentation_tif(tif_file)

        if cell_count is not None:
            # Calculate actual field area
            area_um2, width_um, height_um = calculate_field_area(width, height, pixel_size_um)
            area_mm2 = area_um2 / 1e6  # Convert to mm²

            # Calculate cell density
            cell_density_per_mm2 = cell_count / area_mm2 if area_mm2 > 0 else 0

            total_cells_in_folder += cell_count
            total_area_um2 += area_um2

            print(f"  Cells: {cell_count}")
            print(f"  Image size: {width} x {height} pixels")
            print(f"  Field size: {width_um:.1f} x {height_um:.1f} μm")
            print(f"  Field area: {area_mm2:.4f} mm²")
            print(f"  Cell density: {cell_density_per_mm2:.1f} cells/mm²")

            # Store detailed results
            detailed_results.append({
                'experiment': experiment,
                'pressure': pressure,
                'magnification': magnification,
                'folder': folder,
                'file': file_name,
                'cell_count': cell_count,
                'width_pixels': width,
                'height_pixels': height,
                'pixel_size_um': pixel_size_um,
                'field_area_um2': area_um2,
                'field_area_mm2': area_mm2,
                'cell_density_per_mm2': cell_density_per_mm2,
                'file_path': tif_file
            })
        else:
            print(f"  Failed to process {file_name}")

    # Calculate folder totals and averages
    if total_area_um2 > 0:
        total_area_mm2 = total_area_um2 / 1e6
        average_density = total_cells_in_folder / total_area_mm2

        area_analysis.append({
            'folder': folder,
            'experiment': experiment,
            'pressure': pressure,
            'magnification': magnification,
            'total_cells': total_cells_in_folder,
            'total_area_mm2': total_area_mm2,
            'average_cell_density_per_mm2': average_density,
            'num_images': len([f for f in tif_files if count_cells_in_segmentation_tif(f)[0] is not None])
        })

        print(f"\nFolder Summary:")
        print(f"  Total cells: {total_cells_in_folder}")
        print(f"  Total area: {total_area_mm2:.4f} mm²")
        print(f"  Average density: {average_density:.1f} cells/mm²")

# Create comprehensive analysis
if detailed_results:
    detailed_df = pd.DataFrame(detailed_results)
    area_df = pd.DataFrame(area_analysis)

    print("\n" + "="*80)
    print("COMPREHENSIVE MAGNIFICATION-AWARE ANALYSIS")
    print("="*80)

    # Summary by experimental condition
    print("\n1. RAW COUNTS BY CONDITION:")
    raw_summary = area_df[['experiment', 'magnification', 'total_cells', 'total_area_mm2', 'average_cell_density_per_mm2']]
    print(raw_summary.to_string(index=False))

    # Density comparison (the most meaningful metric)
    print("\n2. CELL DENSITY COMPARISON (cells/mm²):")
    density_comparison = area_df.groupby(['experiment'])['average_cell_density_per_mm2'].agg(['mean', 'std', 'count']).round(1)
    print(density_comparison)

    # Statistical comparison between experiments
    flow3_densities = detailed_df[detailed_df['experiment'] == 'flow3']['cell_density_per_mm2'].values
    pa_densities = detailed_df[detailed_df['experiment'] == '1.4Pa']['cell_density_per_mm2'].values

    if len(flow3_densities) > 0 and len(pa_densities) > 0:
        print(f"\n3. STATISTICAL COMPARISON:")
        print(f"Flow3 (0Pa) density: {np.mean(flow3_densities):.1f} ± {np.std(flow3_densities):.1f} cells/mm² (n={len(flow3_densities)})")
        print(f"1.4Pa density: {np.mean(pa_densities):.1f} ± {np.std(pa_densities):.1f} cells/mm² (n={len(pa_densities)})")

        density_ratio = np.mean(pa_densities) / np.mean(flow3_densities)
        print(f"Density ratio (1.4Pa/Flow3): {density_ratio:.2f}")

        # Simple t-test indication
        from scipy import stats
        try:
            t_stat, p_value = stats.ttest_ind(pa_densities, flow3_densities)
            print(f"T-test p-value: {p_value:.4f}")
        except:
            print("Could not perform t-test (scipy not available)")

    # Magnification-specific analysis
    print(f"\n4. MAGNIFICATION-SPECIFIC INSIGHTS:")
    mag_summary = detailed_df.groupby('magnification').agg({
        'cell_density_per_mm2': ['mean', 'std', 'count'],
        'field_area_mm2': ['mean', 'std']
    }).round(4)
    print(mag_summary)

    # Expected vs observed field sizes
    print(f"\n5. FIELD OF VIEW VALIDATION:")
    for mag in ['x20', 'x40']:
        mag_data = detailed_df[detailed_df['magnification'] == mag]
        if not mag_data.empty:
            observed_fov = mag_data['field_area_mm2'].mean()
            theoretical_fov = MAGNIFICATION_PARAMS[mag]['theoretical_fov_mm2']
            print(f"{mag}: Observed FOV = {observed_fov:.4f} mm², Theoretical = {theoretical_fov:.4f} mm²")

    # Save results
    output_path_detailed = os.path.join(base_path, "cell_analysis_detailed_magnification_aware.csv")
    output_path_summary = os.path.join(base_path, "cell_analysis_summary_by_area.csv")

    detailed_df.to_csv(output_path_detailed, index=False)
    area_df.to_csv(output_path_summary, index=False)

    print(f"\nResults saved:")
    print(f"  Detailed: {output_path_detailed}")
    print(f"  Summary: {output_path_summary}")

    # Recommendations
    print(f"\n" + "="*80)
    print("ANALYSIS RECOMMENDATIONS:")
    print("="*80)
    print("1. Use CELL DENSITY (cells/mm²) for comparing between conditions")
    print("2. Raw cell counts are biased by different field sizes between magnifications")
    print("3. x40 magnification provides higher resolution but smaller coverage area")
    print("4. Consider pooling data by experiment rather than by magnification")
    print("5. Report both individual image densities and experiment-level averages")

else:
    print("\nNo TIF files found or processed successfully.")
    print("Please check:")
    print(f"1. Base path: {base_path}")
    print(f"2. Subfolders: {folders}")
    print(f"3. Target subfolder: {subfolder_name}")
    print("4. Ensure TIF files exist in the Segmented/Cell_merged_conservative subfolders")
    print("5. Check if TIF files are properly labeled segmentation masks")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.
Starting magnification-aware cell counting from TIF segmentation files...

Magnification parameters:
  x20: pixel size = 0.325 μm, theoretical FOV = 1.0 mm²
  x40: pixel size = 0.1625 μm, theoretical FOV = 0.25 mm²

Processing folder: flow3-x20
Path: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3-x20/Cell_merged_conservative
Found 8 TIF file(s)

Processing: 0Pa_U_05mar19_20x_L2RA_Flat_seq001_cell_mask_merged_conservative.tif
  Cells: 329
  Image size: 1024 x 1024 pixels
  Field size: 332.8 x 332.8 μm
  Field area: 0.1108 mm²
  Cell density: 2970.5 cells/mm²

Processing: 0Pa_U_05mar19_20x_L2RA_Flat_seq002_cell_mask_merged_conservative.tif
  Cells: 368
  Image size: 1024 x 1024 pixels
  Field size: 332.8 x 332.8 μm
  Field area: 0.1108 mm²
  Cell density: 3322.6 cells/mm²

Processing: 0Pa_U_05mar19_2

In [5]:
import pandas as pd
import numpy as np
import os
import glob
from google.colab import drive
from PIL import Image
import cv2

# Mount Google Drive
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    exit()

def count_cells_in_segmentation_tif(tif_path):
    """
    Count the number of segmented cells in a TIF file.
    Assumes segmentation masks where each cell has a unique label/intensity.
    """
    try:
        # Try reading with PIL first
        img = Image.open(tif_path)
        img_array = np.array(img)

        # If it's a labeled segmentation mask, count unique non-zero labels
        unique_labels = np.unique(img_array)
        # Remove background (assuming background is 0)
        cell_labels = unique_labels[unique_labels > 0]
        cell_count = len(cell_labels)

        # Get image dimensions for reference
        height, width = img_array.shape[:2]

        return cell_count, width, height, img_array.dtype

    except Exception as e:
        # Fallback to OpenCV
        try:
            img_array = cv2.imread(tif_path, cv2.IMREAD_ANYDEPTH)
            if img_array is None:
                return None, None, None, None

            unique_labels = np.unique(img_array)
            cell_labels = unique_labels[unique_labels > 0]
            cell_count = len(cell_labels)

            height, width = img_array.shape[:2]
            return cell_count, width, height, img_array.dtype

        except Exception as e2:
            print(f"Error reading {tif_path}: {e}, {e2}")
            return None, None, None, None

# Define the base path and folder names
base_path = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/"
folders = ["flow3-x20", "1.4Pa-x40", "1.4Pa-x20"]

# Fixed: Remove the leading slash from subfolder_name
subfolder_name = "Cell_merged_conservative"

# Lists to store results
detailed_results = []
summary_results = []

print("Starting cell counting from TIF segmentation files...")

# Loop through each experimental folder
for folder in folders:
    folder_path = os.path.join(base_path, folder, subfolder_name)
    print(f"\n{'='*60}")
    print(f"Processing folder: {folder}")
    print(f"Path: {folder_path}")

    if not os.path.exists(folder_path):
        print(f"Warning: Path '{folder_path}' not found. Skipping.")
        # Let's also check what's actually in the parent directory
        parent_path = os.path.join(base_path, folder)
        if os.path.exists(parent_path):
            print(f"Contents of {parent_path}:")
            try:
                contents = os.listdir(parent_path)
                for item in contents:
                    print(f"  - {item}")
            except Exception as e:
                print(f"  Could not list contents: {e}")
        continue

    # Find all TIF files
    tif_files = glob.glob(os.path.join(folder_path, "*.tif")) + glob.glob(os.path.join(folder_path, "*.tiff"))

    if not tif_files:
        print(f"No TIF files found in {folder_path}")
        # List what files are actually there
        try:
            all_files = os.listdir(folder_path)
            print(f"Files found in directory: {all_files}")
        except Exception as e:
            print(f"Could not list files: {e}")
        continue

    print(f"Found {len(tif_files)} TIF file(s)")

    # Extract experiment parameters
    if "flow3" in folder:
        pressure = '0Pa'
        experiment = 'flow3'
    elif "1.4Pa" in folder:
        pressure = '1.4Pa'
        experiment = '1.4Pa'
    else:
        pressure = 'Unknown'
        experiment = 'Unknown'

    if "x40" in folder:
        magnification = 'x40'
    elif "x20" in folder:
        magnification = 'x20'
    else:
        magnification = 'unknown'

    total_cells_in_folder = 0
    successful_files = 0

    # Process each TIF file
    for tif_file in tif_files:
        file_name = os.path.basename(tif_file)
        print(f"\nProcessing: {file_name}")

        cell_count, width, height, dtype = count_cells_in_segmentation_tif(tif_file)

        if cell_count is not None:
            total_cells_in_folder += cell_count
            successful_files += 1

            print(f"  Cells: {cell_count}")
            print(f"  Image size: {width} x {height} pixels")

            # Store detailed results
            detailed_results.append({
                'experiment': experiment,
                'pressure': pressure,
                'magnification': magnification,
                'folder': folder,
                'file': file_name,
                'cell_count': cell_count,
                'width_pixels': width,
                'height_pixels': height,
                'file_path': tif_file
            })
        else:
            print(f"  Failed to process {file_name}")

    # Calculate folder summary
    if successful_files > 0:
        average_cells_per_image = total_cells_in_folder / successful_files

        summary_results.append({
            'folder': folder,
            'experiment': experiment,
            'pressure': pressure,
            'magnification': magnification,
            'total_cells': total_cells_in_folder,
            'num_images': successful_files,
            'average_cells_per_image': average_cells_per_image
        })

        print(f"\nFolder Summary:")
        print(f"  Total cells: {total_cells_in_folder}")
        print(f"  Number of images: {successful_files}")
        print(f"  Average cells per image: {average_cells_per_image:.1f}")

# Create comprehensive analysis
if detailed_results:
    detailed_df = pd.DataFrame(detailed_results)
    summary_df = pd.DataFrame(summary_results)

    print("\n" + "="*80)
    print("COMPREHENSIVE CELL COUNT ANALYSIS")
    print("="*80)

    # Summary by experimental condition
    print("\n1. RAW COUNTS BY CONDITION:")
    summary_display = summary_df[['experiment', 'magnification', 'total_cells', 'num_images', 'average_cells_per_image']]
    print(summary_display.to_string(index=False))

    # Cell count comparison by experiment
    print("\n2. CELL COUNT COMPARISON BY EXPERIMENT:")
    count_comparison = summary_df.groupby(['experiment']).agg({
        'total_cells': 'sum',
        'num_images': 'sum',
        'average_cells_per_image': ['mean', 'std']
    }).round(1)
    print(count_comparison)

    # Statistical comparison between experiments
    flow3_counts = detailed_df[detailed_df['experiment'] == 'flow3']['cell_count'].values
    pa_counts = detailed_df[detailed_df['experiment'] == '1.4Pa']['cell_count'].values

    if len(flow3_counts) > 0 and len(pa_counts) > 0:
        print(f"\n3. STATISTICAL COMPARISON (cells per image):")
        print(f"Flow3 (0Pa): {np.mean(flow3_counts):.1f} ± {np.std(flow3_counts):.1f} cells/image (n={len(flow3_counts)})")
        print(f"1.4Pa: {np.mean(pa_counts):.1f} ± {np.std(pa_counts):.1f} cells/image (n={len(pa_counts)})")

        # Simple t-test indication
        from scipy import stats
        try:
            t_stat, p_value = stats.ttest_ind(pa_counts, flow3_counts)
            print(f"T-test p-value: {p_value:.4f}")
        except:
            print("Could not perform t-test (scipy not available)")

    # Magnification-specific analysis
    print(f"\n4. MAGNIFICATION-SPECIFIC ANALYSIS:")
    mag_summary = detailed_df.groupby('magnification').agg({
        'cell_count': ['mean', 'std', 'count', 'sum']
    }).round(1)
    print(mag_summary)

    # Individual image details
    print(f"\n5. INDIVIDUAL IMAGE DETAILS:")
    for exp in detailed_df['experiment'].unique():
        exp_data = detailed_df[detailed_df['experiment'] == exp]
        print(f"\n{exp} experiment:")
        for _, row in exp_data.iterrows():
            print(f"  {row['file']}: {row['cell_count']} cells ({row['magnification']})")

    # Save results
    output_path_detailed = os.path.join(base_path, "cell_count_detailed.csv")
    output_path_summary = os.path.join(base_path, "cell_count_summary.csv")

    detailed_df.to_csv(output_path_detailed, index=False)
    summary_df.to_csv(output_path_summary, index=False)

    print(f"\nResults saved:")
    print(f"  Detailed: {output_path_detailed}")
    print(f"  Summary: {output_path_summary}")

    # Recommendations
    print(f"\n" + "="*80)
    print("ANALYSIS NOTES:")
    print("="*80)
    print("1. Raw cell counts are reported without density calculations")
    print("2. Different magnifications will have different field sizes")
    print("3. For density calculations, you'll need accurate pixel size calibration")
    print("4. Consider the field of view differences when comparing magnifications")
    print("5. Average cells per image gives a sense of cell density within each condition")

else:
    print("\nNo TIF files found or processed successfully.")
    print("Please check:")
    print(f"1. Base path: {base_path}")
    print(f"2. Subfolders: {folders}")
    print(f"3. Target subfolder: {subfolder_name}")
    print("4. Ensure TIF files exist in the Segmented/Cell_merged_conservative subfolders")
    print("5. Check if TIF files are properly labeled segmentation masks")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.
Starting cell counting from TIF segmentation files...

Processing folder: flow3-x20
Path: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3-x20/Cell_merged_conservative
Found 8 TIF file(s)

Processing: 0Pa_U_05mar19_20x_L2RA_Flat_seq001_cell_mask_merged_conservative.tif
  Cells: 329
  Image size: 1024 x 1024 pixels

Processing: 0Pa_U_05mar19_20x_L2RA_Flat_seq002_cell_mask_merged_conservative.tif
  Cells: 368
  Image size: 1024 x 1024 pixels

Processing: 0Pa_U_05mar19_20x_L2RA_Flat_seq003_cell_mask_merged_conservative.tif
  Cells: 385
  Image size: 1024 x 1024 pixels

Processing: 1.4Pa_U_05mar19_20x_L2R_Flat_seq001_cell_mask_merged_conservative.tif
  Cells: 264
  Image size: 1024 x 1024 pixels

Processing: 1.4Pa_U_05mar19_20x_L2R_Flat_seq002_cell_mask_merged_conservative.tif
  Cells: 210
  Image size: 